# Grading
1. 8/8
2. 4/4 
3. 5/5
4. 3/3  
Awesome work!  

In [1]:
from pylab import *
import timeit

## Problem 1
The formula for the present value of the cash flows $C_1, \dots, C_n$ is $$\sum_{i=1}^n \frac{C_i}{(1+r)^i}.$$

In [2]:
# Initial data
C =  120.0 * arange(500,1200)
r = 0.01

### A: Python loop

In [3]:
def for_loop(Cash, rate):
    sum = 0
    x = 1/(1 + rate)
    for i in arange(0, len(Cash)):
        sum = sum + C[i]*(x**(i+1)) # the array C starts at 0, so the exponent is one higher
    return sum

t = timeit.Timer('for_loop(C,r)', 'from __main__ import for_loop, C, r')
print("1000 evaluations of this function take", t.timeit(number=1000), "s")
print("The present value of the given cashflow is {0:0.2f} €".format(for_loop(C,r)))

1000 evaluations of this function take 2.001204364000003 s
The present value of the given cashflow is 7185271.35 €


### B: Horner's scheme

In [4]:
def horners(Cash, rate):
    sum = 0
    x = 1/(1 + rate)
    for i in arange(0, len(Cash)):
        sum = sum + Cash[i]*x
        x = x/(1 + rate)
    return sum

t = timeit.Timer('horners(C,r)', 'from __main__ import horners, C, r')
print("1000 evaluations of this function take", t.timeit(number=1000), "s")
print("The present value of the given cashflow is {0:0.2f} €".format(horners(C,r)))

1000 evaluations of this function take 0.3770770289999916 s
The present value of the given cashflow is 7185271.35 €


### C: The `polyval` function

In [5]:
def my_polyval(Cash, rate):
    x = 1/(1 + rate)
    return x*polyval(Cash[::-1], x) # Reversing the cashflow array gives polyval the correct order of terms

t = timeit.Timer('my_polyval(C,r)', 'from __main__ import my_polyval, C, r')
print("1000 evaluations of this function take", t.timeit(number=1000), "s")
print("The present value of the given cashflow is {0:0.2f} €".format(my_polyval(C,r)))

1000 evaluations of this function take 0.289320883000002 s
The present value of the given cashflow is 7185271.35 €


### D: Dot product

In [6]:
def dot_product(Cash, rate):
    i = arange(1, len(Cash)+1)
    x = 1/(1 + rate)
    return dot(Cash, x**i)

t = timeit.Timer('dot_product(C,r)', 'from __main__ import dot_product, C, r')
print("1000 evaluations of this function take", t.timeit(number=1000), "s")
print("The present value of the given cashflow is {0:0.2f} €".format(dot_product(C,r)))

1000 evaluations of this function take 0.02381686200000388 s
The present value of the given cashflow is 7185271.35 €


## Problem 2
See hand-in

## Problem 3

In [7]:
def amortization_scheduler(P, r, m, n):
    # solve for monthly payments from formula for PV of a general annuity
    monthly_payment = P*r/(m* (1 - (1 + r/m)**(-m*n)))
    effective_rate = (1 + r/m)**m - 1
    
    months_paid = arange(1, m*n + 1)
    
    def remaining_principal(k):
        # present value of remaining future cashflow after k payments
        return monthly_payment*m*(1 - (1 + r/m)**(-n*m + k))/r
    
    def interest_payment(k):
        # period interest rate times remaining principal from past period
        return remaining_principal(k-1)*r/m
    
    def principal_payment(k):
        return monthly_payment - interest_payment(k)
    
    print("Month", "Interest Payment", "Principal Paid", "Remaining Principal")
    for i in months_paid:
        print('%-5i' % i, '%-16.2f' % interest_payment(i), '%-16.2f' % principal_payment(i), '%-16.2f' %remaining_principal(i))
    
    print("The expected monthly payment is {0:0.2f}".format(monthly_payment))
    print("The effective interest rate is {0:0.5f}".format(effective_rate))

amortization_scheduler(500000, 0.02, 12, 20)

Month Interest Payment Principal Paid Remaining Principal
1     833.33           1696.08          498303.92       
2     830.51           1698.91          496605.01       
3     827.68           1701.74          494903.26       
4     824.84           1704.58          493198.69       
5     822.00           1707.42          491491.27       
6     819.15           1710.26          489781.00       
7     816.30           1713.12          488067.89       
8     813.45           1715.97          486351.92       
9     810.59           1718.83          484633.09       
10    807.72           1721.69          482911.39       
11    804.85           1724.56          481186.83       
12    801.98           1727.44          479459.39       
13    799.10           1730.32          477729.07       
14    796.22           1733.20          475995.87       
15    793.33           1736.09          474259.78       
16    790.43           1738.98          472520.80       
17    787.53           1741.88

## Problem 4
The internal rate of return of a given cashflow $C_1, \cdots, C_n$ with price $P$ is the value of $r$ that solves the equation
$$P = \sum_{i=1}^n\frac{C_i}{(1+r)^i}.$$

In [8]:
# Initial data
C = 120.0 * arange(42,52)
P = 50000.0

from scipy import optimize

def irr_finder(Cashflow, Price):
    i = arange(1, len(Cashflow) + 1)
    
    def f(rate):
        # Returns function whose root is the IRR
        return dot(Cashflow, 1/(1 + rate)**i) - Price
    
    return optimize.brentq(f, 0, 1)
        
print("The internal rate of return is {0:0.2f}%".format(irr_finder(C,P)*100))

The internal rate of return is 1.98%
